In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import time
import sys
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import math
from time import gmtime, strftime

# 설정 파일
import config as conf

In [2]:
def drawProgressBar(percent, barLen = 20):
    # percent float from 0 to 1. 
    sys.stdout.write("\r")
    sys.stdout.write("[{:<{}}] {:.0f}%".format("=" * int(barLen * percent), barLen, percent * 100))
    sys.stdout.flush()

In [3]:
# 예측 유저 목록
predict_users = pd.read_csv(conf.predict_res, names=['user_id'])

In [4]:
# 메타 데이터
metadata = pd.read_json('res/metadata.json', lines=True)

In [5]:
# joy
after_oct_articles = []
after_oct_articles = metadata[metadata.reg_ts >= 1538352000000]

In [6]:
# 유저 데이터
users = pd.read_json('res/users.json', lines=True)

In [ ]:
# read 데이터
read_file_lst = glob.glob('res/read/*')
exclude_file_lst = ['read.tar']

read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
        
read = pd.concat(read_df_lst)

In [ ]:
# read 데이터 가공
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

In [ ]:
# 글별 소비수 통계
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']

In [ ]:
author_read_dic = {}

for index, row in atc_read_cnt.iterrows():
    atc = row['article_id']
    author = ((row['article_id']).split("_"))[0]
    if author in author_read_dic:
        author_read_dic[author].append(atc)
    else:
        article_list = []
        article_list.append(atc)
        author_read_dic[author] = article_list

In [ ]:
atc = metadata.copy()
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']

In [11]:
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]

In [12]:
read_cnt_frame = atc_read_cnt_nn.sort_values(["read_cnt"], ascending=[False])
optimize_frame = read_cnt_frame.drop(['id', 'display_url', 'sub_title', 'magazine_id', 'reg_ts', 'title', 'author_id', 'reg_dt', 'type'], axis=1)

In [13]:
# article 기반 데이터 정제
article_detail_dic = {}
for row in optimize_frame.values:
    article_detail_dic[row[0]] = {'read_cnt': row[1], 'keyword': row[2] ,'datetime': row[3]}

In [14]:
# 전체 유저별 읽은 글 목록
user_read_dic = {}
for row in read_raw.values:
    user_id = row[2]
    article = row[3]
    if user_read_dic.get(user_id, "empty") == "empty":
        user_read_dic[user_id] = [article]
    else:
        user_read_dic[user_id].append(article)

In [15]:
#Joy 유저별 팔로우 목록
user_follow_list = []
for row in users.values:
    user_id = row[1]
    follow_list = row[0]
    if "@brunch" in follow_list: follow_list.remove("@brunch")

    if len(follow_list) > 1:
        user_follow_list.append(follow_list)

In [16]:
# 유저별 팔로우 목록
user_follow_dict = {}
for row in users.values:
    user_id = row[1]
    follow_list = row[0]
    user_follow_dict[user_id] = follow_list

In [42]:
# 유저의 follow가 아니면서 읽은 글 수가 많은 작가 추리기 : 전체 유저가 읽은 목록 대상 조회
followable_dic = {}

for user_id, value in user_read_dic.items():
    arcs = list(value).copy()
    authors_map = map(lambda x: (x.split("_"))[0], arcs)
    authors = list(authors_map).copy()
    
    orinC1 = set(authors)
    C1 = list(orinC1).copy()
    
    dic = {}
    for at in C1:
        dic[at] = authors.count(at)
    
    five_percent = round(len(user_read_dic[user_id]) / 5)
    
    for k, v in dic.items():
        if v > 10: #or v > five_percent:
            # 팔로우하는 작가가 있는 유저
            if user_id in user_follow_dict:
                if not k in (user_follow_dict[user_id]):
                    if user_id in followable_dic:
                        followable_dic[user_id].append(k)
                    else:
                        auth_list = []
                        auth_list.append(k)
                        followable_dic[user_id] = auth_list
            else:
                if user_id in followable_dic:
                    followable_dic[user_id].append(k)
                else:
                    auth_list = []
                    auth_list.append(k)
                    followable_dic[user_id] = auth_list

In [19]:
def convertTime (dateTime):
    if dateTime == 0:
        return 0
    t = pd.Timestamp(dateTime)
    return time.mktime(t.timetuple())

In [20]:
read_cnt_max = max(read_cnt_frame.read_cnt)
reg_datetime_max = max(read_cnt_frame[read_cnt_frame.reg_datetime < '2090-12-31'].reg_datetime)
reg_datetime_max = convertTime(reg_datetime_max)

In [21]:
# 유저별 팔로우 목록
user_follow_dict = {}
for row in users.values:
    user_id = row[1]
    follow_list = row[0]
    user_follow_dict[user_id] = follow_list

In [22]:
# 읽은 글에 대한 팔로우 가중치 계산에 사용
def to_timeScore(dt_time):
    return time.mktime(datetime.strptime(dt_time[6:8] +"/"+dt_time[4:6]+"/"+dt_time[:4], "%d/%m/%Y").timetuple()) * (0.2 / reg_datetime_max)

In [23]:
def getKeywordTimeScore(datetime):
    return convertTime(data['datetime']) * (0.2 / reg_datetime_max) #최근 글 일수록 가중치

In [24]:
# 최근 읽은 글일 수록 가중치 추가
def getArticleTimeScore(article, userId):
    if article_detail_dic.get(article, "empty") == "empty":
        return to_timeScore(user_last_read_dic[userId][article])
    
    data = article_detail_dic[article]
    return convertTime(data['datetime']) * (0.3 / reg_datetime_max) #최근 글 일수록 가중치

In [25]:
def getArticleScore(article):
    data = article_detail_dic[article]
    cnt_p = data['read_cnt'] * (0.02 / read_cnt_max) # 읽은 사람이 많을 수록 가중치
    time_p = convertTime(data['datetime']) * (0.35 / reg_datetime_max) #최근 글 일수록 가중치
    time_p = time_p if time_p <= 0.35 else 0
    return cnt_p + time_p

In [26]:
# 키워드에 대한 상관관계 파악
user_keyword_article_read_dic = {}
for id in user_read_dic:
    article_read_dic = {}
    read_articles = user_read_dic[id]
    for article in read_articles:
        if article_detail_dic.get(article, "empty") == "empty":
            continue
        data = article_detail_dic[article]
        for keyword in data['keyword']:
            if (article_read_dic.get(keyword, "empty") == "empty"):
                article_read_dic[keyword] = getKeywordTimeScore(data['datetime'])
            else:
                article_read_dic[keyword] += getKeywordTimeScore(data['datetime'])
           
    user_keyword_article_read_dic[id] = article_read_dic

In [27]:
def getKeywordRanking (id):
    if (user_keyword_article_read_dic.get(id, "empty") == "empty") | (len(user_keyword_article_read_dic[id]) == 0):
        return {}
    rank_dic = dict(sorted(user_keyword_article_read_dic[id].items(), key=lambda x:x[1], reverse = True))
    rank_max = max(rank_dic.values())
    p1 = 0.3 / rank_max # 최대 가중치 0.3
    
    result = {}
    for item in rank_dic:
        result[item] = rank_dic[item] * p1
    return result

In [28]:
def getKeywordAvgScore (keyword_dic, article):
    if article_detail_dic.get(article, "empty") == "empty":
        return 0
    result = 0
    cnt = 0
    keyword_list = article_detail_dic[article]['keyword']
    for keyword in keyword_list:
        if keyword_dic.get(keyword, "empty") == "empty":
            continue
        result += keyword_dic[keyword]
        cnt += 1
    return result if result == 0 else result / cnt

In [29]:
def getKeywordMaxScore (keyword_dic, article):
    # 키워드 정보 없음
    if article_detail_dic.get(article, "empty") == "empty":
        return 0
    result = []
    keyword_list = article_detail_dic[article]['keyword']
    for keyword in keyword_list:
        if keyword_dic.get(keyword, "empty") == "empty":
            continue
        result.append(keyword_dic[keyword])
    return max(result) if len(result) > 0 else 0

In [30]:
user_last_read_dic = {}
for row in read_raw.values:
    if user_last_read_dic.get(row[2], "empty") == "empty":
        user_last_read_dic[row[2]] = {}
    user_last_read_dic[row[2]][row[3]] = row[0]

In [31]:
# 특정 유저마다 팔로우 하고있는 작가별 읽은 글의 수
user_follow_article_read_dic = {}
for id in user_read_dic:
    article_read_dic = {}
    read_articles = user_read_dic[id]
    for article in read_articles:
        follow_id = article.split('_')[0]
        # 팔로우가 없는 경우
        if (user_follow_dict.get(id, "empty") == "empty"):
            user_follow_dict[id] = []
            continue
            
        if (follow_id in user_follow_dict[id]):
            if (article_read_dic.get(follow_id, "empty") == "empty"):
                article_read_dic[follow_id] = getArticleTimeScore(article, id);
            else:
                article_read_dic[follow_id] += getArticleTimeScore(article, id)
    user_follow_article_read_dic[id] = article_read_dic

In [32]:
def getFollowingRanking (id):
    if (user_follow_article_read_dic.get(id, "empty") == "empty") | (len(user_follow_article_read_dic[id]) == 0):
        return {}
    rank_dic = dict(sorted(user_follow_article_read_dic[id].items(), key=lambda x:x[1], reverse = True))
    rank_max = max(rank_dic.values())
    p1 = 0.4 / rank_max # 최대 가중치 0.4
    
    result = {}
    for item in rank_dic:
        result[item] = rank_dic[item] * p1
    return result

In [33]:
user_keyword_ranking_dic = {}
for id in predict_users.user_id:
    user_keyword_ranking_dic[id] = getKeywordRanking(id)

In [35]:
# followable 얻기위해 apriori 알고리즘 사용
print("[Train-1] Start")
test_list = user_follow_list.copy()

count = len(test_list)

def createC1(dataSet):
    C1=[]
    
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])

    C1.sort()
    return map(frozenset, C1)


def scanD(D, Ck, minSupport):
    # 요소 n 은 몇개의 그룹에 포함되어 있는지 계산한다.
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can] = 1 
                else: ssCnt[can] += 1

    
    # 지지도가 0.5보다 높은것들의 리스트를 구함. 
    numItems = float(count) # 그룹 갯수
    retList = []
    supportData = {}
    for key , value in ssCnt.items():
        try:
            support = value / numItems
            
            if support >= minSupport:
                retList.insert(0,key)
            
            supportData[key] = support
        except ZeroDivisionError:
            print('zero_division_error')
        
        

    return retList, supportData

def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])

    return retList

# 특정 지지도 이상의 값들의 쌍을 찾음 
def apriori(dataset, minSupport = 0.01):
    orinC1 = createC1(dataset)
    C1 = list(orinC1).copy()
    
    orinD = map(set, dataset)
    D = list(orinD).copy()

    L1 , supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2],k)
        Lk,supK = scanD(D,Ck,minSupport) # 후보그룹을 모두 찾는다.
        supportData.update(supK)
        L.append(Lk) #이게 핵심!특정 지지도 이상의 그룹들만 L에 담는다.즉 가지치기
        k += 1
        
    return L, supportData

def generateRules(L, supportData, minConf=0.01):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i>1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet,H1,supportData, bigRuleList, minConf)

    return bigRuleList


def calcConf(freqSet, H, supportData, br1, minConf=0.01):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet-conseq]
        if conf >= minConf:
            #print (freqSet-conseq, '-->', conseq, 'conf:', conf)
            
            br1.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.01):
    m = len(H[0])
    
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)

        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)
            
L, suppData = apriori(test_list, minSupport = 0.01)
# print ("L:" + str(L))
# print (".........................")
# print ("suppData:" + str(suppData))
rules = generateRules(L, suppData, minConf=0.01)


[Train-1] Start
frozenset({'@varo'}) --> frozenset({'@kangsunseng'}) conf: 0.23713202094997293
frozenset({'@kangsunseng'}) --> frozenset({'@varo'}) conf: 0.3383148673022417
frozenset({'@varo'}) --> frozenset({'@goodrain'}) conf: 0.23063030521943292
frozenset({'@goodrain'}) --> frozenset({'@varo'}) conf: 0.3730645632486123
frozenset({'@ebprux'}) --> frozenset({'@roysday'}) conf: 0.30459551842005317
frozenset({'@roysday'}) --> frozenset({'@ebprux'}) conf: 0.19615996086584322
frozenset({'@ebprux'}) --> frozenset({'@mobiinside'}) conf: 0.23756171667299658
frozenset({'@mobiinside'}) --> frozenset({'@ebprux'}) conf: 0.26611359285258457
frozenset({'@ebprux'}) --> frozenset({'@ideawriter'}) conf: 0.23585263957462968
frozenset({'@ideawriter'}) --> frozenset({'@ebprux'}) conf: 0.6076320939334638
frozenset({'@roysday'}) --> frozenset({'@dryjshin'}) conf: 0.18001712119359178
frozenset({'@dryjshin'}) --> frozenset({'@roysday'}) conf: 0.17601339232332894
frozenset({'@jidesign'}) --> frozenset({'@soo

In [39]:
relative_dic = {}
for rule in rules:
    conf = rule[2]
    if conf > 0.3:
        user = list(rule[0])[0]
        target = list(rule[1])[0]
        if user in relative_dic:
            relative_dic[user].append(target)
        else:
            target_list = []
            target_list.append(target)
            relative_dic[user] = target_list

[====================] 100%

In [43]:
# followable_dic 목록에 연관성이 30%이상인 팔로워 목록을 추가한다
step = 0
step_end = len(user_follow_dict.items())
for key, follower_list in user_follow_dict.items():
    step +=1
    drawProgressBar(step / step_end)
    avail = []
    
    for follower in follower_list:
        if follower in relative_dic:
            avail.extend(relative_dic[follower])
    
    if len(avail) > 0:
        #print(str(avail))
        for author in avail:
            if not author in follower_list:
                if not key in followable_dic:
                    #print('make new list: ' + key)
                    new_list = []
                    followable_dic[key] = new_list
                    
                followable_dic[key].append(author)
                #print('added \'' + author + '\' in ' + key)

[=================== ] 100%                   ] 2%[=                   ] 6%[==                  ] 10%[======              ] 32%[=======             ] 35%[=======             ] 40%[===========         ] 58%[==============      ] 74%[===============     ] 75%[===============     ] 77%[===============     ] 79%[=================   ] 90%

In [44]:
# 키워드 목록
keyword_list = list(after_oct_articles.keyword_list)
ids = list(after_oct_articles.id)
size = len(after_oct_articles)

key_dic = {}
for i in range(0, size):
    if len(keyword_list[i]) > 0:
        key_dic[ids[i]] = keyword_list[i]

In [47]:
# 많이 읽은 키워드 정리
step = 0
step_end = len(user_read_dic.items())

user_keywords_dic = {}

for user_id, value in user_read_dic.items():
    step +=1
    drawProgressBar(step / step_end)
    
    arcs = list(value).copy()
    has_key_cnt = 0
    dic = {}
    for arc in arcs:
        # 매거진 글인지 체크
        if arc in key_dic:
            has_key_cnt += 1 # 키워드가 있는 글을 몇 개나 읽었나
            key_list = key_dic[arc]
            
            for key in key_list:
                if not key in dic:
                    dic[key] = 0
                dic[key] += 1 # 읽은 글의 키워드 카운팅
    
    for k, v in dic.items():
        if v < 10: continue
        score = v / has_key_cnt
        # print(k + ' : ' + str(score))
        
        if not user_id in user_keywords_dic:
            score_dic = {}
            user_keywords_dic[user_id] = score_dic
        
        (user_keywords_dic[user_id])[k] = score

[====================] 100%                   ] 4%[=                   ] 8%[=====               ] 26%[=======             ] 38%[==========          ] 50%[===============     ] 78%[===============     ] 79%[================    ] 81%[=================   ] 89%

In [48]:
# 매거진 정리
has_mag_atcs = after_oct_articles[after_oct_articles.magazine_id > 0]

In [49]:
ids = list(has_mag_atcs.id)
mags = list(has_mag_atcs.magazine_id)

size = len(has_mag_atcs)
mags_dic = {}

for i in range(0, size):
    mags_dic[ids[i]] = mags[i]

In [50]:
# 유저의 매거진 글 선호도 체크
user_mag_dic = {} # dic[dic[int]] 객체

step = 0
step_end = len(user_read_dic.items())

for user_id, value in user_read_dic.items():
    step +=1
    drawProgressBar(step / step_end)
    
    arcs = list(value).copy()
    
    dic = {}
    for arc in arcs:
        # 매거진 글인지 체크
        if arc in mags_dic:
            mag_id = mags_dic[arc]
            if not mag_id in dic:
                dic[mag_id] = 0
            dic[mag_id] += 1 # 읽은 매거진 글 카운팅
    
    for k, v in dic.items():
        arc_cn = mag_count_dic[k]
        prefer = v / arc_cn
        # print('arc_cn: ' + str(arc_cn) + ': ' + str(v))
        
        if not user_id in user_mag_dic:
            prefer_dic = {}
            user_mag_dic[user_id] = prefer_dic
            
        # 여러번 읽은 경우에 최대치 1
        if prefer > 1.0:
            prefer = 1.0
        (user_mag_dic[user_id])[k] = prefer

[                    ] 0%

NameError: name 'mag_count_dic' is not defined

In [ ]:
# 유저의 작가별 글 선호도 체크
user_prefer_dic = {} # dic[dic[float]] 객체

step = 0
step_end = len(user_read_dic.items())

for user_id, value in user_read_dic.items():
    step +=1
    drawProgressBar(step / step_end)
    
    arcs = list(value).copy()
    authors_map = map(lambda x: (x.split("_"))[0], arcs)
    authors = list(authors_map).copy()
    
    orinC1 = set(authors)
    C1 = list(orinC1).copy()
    
    dic = {}
    for at in C1:
        if len(at) > 0:
            dic[at] = authors.count(at)
    #print(str(dic))
    
    for k, v in dic.items():
        prefer = v / len(author_read_dic[k]) # 작가가 쓴 글에서 읽은 글 수
        
        if user_id in user_prefer_dic:
            (user_prefer_dic[user_id])[k] = prefer
        else:
            prefer_dic = {}
            prefer_dic[k] = prefer
            user_prefer_dic[user_id] = prefer_dic


In [ ]:
# 포인트 계산 - 이게 오래 걸림

temp = {}
ids = list(after_oct_articles.id)
users = list(after_oct_articles.user_id)
times = list(after_oct_articles.reg_ts)
size = len(after_oct_articles)

for i in range(0, size):
    temp[ids[i]] = {"author": users[i], "ts": times[i]}
    
#print(len(temp))

all_read_cnt = len(all_read)
#print(all_read_cnt)
maxT = 1554044340000
term = maxT - 1538352219000 # 1546300963000
term2 = maxT - 1548979335000 # 2월 이후 글 중에서 

pre_atc_dic = {}
feb_articles_dic = {}

step = 0
step_end = len(temp.items())

for key, value in temp.items():
    step +=1
    drawProgressBar(step / step_end)
    
    author = value['author']
    arc_id = key
    popular = 0
    read_cnt = 0
    
    if (all_atc_read_cnt['article_id'] == arc_id).any():
        a = all_atc_read_cnt[all_atc_read_cnt['article_id'] == arc_id]['read_cnt']
        read_cnt = a.values[0]
        if read_cnt > 10:
            popular = (read_cnt / all_read_cnt) * 1000 # 글의 유명세
            # print(popular)
    
    add_dic = {}
    date = (maxT - value['ts']) / term # 최신에 가까울수록 1

    date2 = 0
    if value['ts'] > 1548979335000: # 2월 이후 애들끼리 경쟁
        date2 = (maxT - value['ts']) / term2 
    
    # 옛날 글 일수록 점수 많이 낮아지라고..
    if date >= 0.1:
        date = math.log10(date * 10)
   
    add_dic['date_point'] = date
    add_dic['popular'] = popular
    add_dic['author'] = author
    # print(str(add_dic))
    
    if value['ts'] > 1548979335000:
        add_dic['date_point'] = date2
        feb_articles_dic[arc_id] = add_dic # 2월 이후 글
    else:
        pre_atc_dic[arc_id] = add_dic # 2월 이전 전체 글


In [ ]:
result_dic = {}
step = 0
step_end = len(predict_users.user_id)

for user in predict_users.user_id:
    step +=1
    drawProgressBar(step / step_end)
    
    arc_dic = {}
   
    is_in_preferdic = False
    if user in user_prefer_dic:
        is_in_preferdic = True
        
    has_follow = False
    if user in user_follow_dict:
        has_follow = True
    
    has_followable = False
    if user in followable_dic:
        has_followable = True
        
    has_mag_dic = False
    if user in user_mag_dic:
        has_mag_dic = True
        
    has_key_dic = False
    if user in user_keywords_dic:
        has_key_dic = True
        
    for key, value in feb_articles_dic.items():
        # print(value)
        author = value['author']
        arc_id = key
        date = value['date_point']
        popular = value['popular']
        
        # 유저가 이미 읽은글인지 판단
        if arc_id in user_read_dic[user]:
            # print('already read')
            continue
        
        au_prefer = 0.0
        if is_in_preferdic:
            if author in user_prefer_dic[user]:
                au_prefer = user_prefer_dic[user][author] # 작가 선호도
        
        mag_prefer = 0.0
        if has_mag_dic:
            if arc_id in mags_dic:
                mag_id = mags_dic[arc_id]
                if mag_id in user_mag_dic[user]:
                    mag_prefer = user_mag_dic[user][mag_id] # 매거진 선호도
        
        keyword_score = 0.0

        if has_key_dic and arc_id in key_dic:
            keys = key_dic[arc_id]
            # print('key_list: ' + str(keys))
            
            for w in keys:
                if w in user_keywords_dic[user]:
                    keyword_score += user_keywords_dic[user][w]
        
        if keyword_score > 1.0:
            keyword_score = 1.0

        base = date + au_prefer + popular + mag_prefer + keyword_score
  
        # 팔로우 작가가 있는 유저
        if has_follow:
            if author in (user_follow_dict[user]): # 팔로우 작가면
                base = base * 3
                # print("follow author " + str(base))
                
        if has_followable:
            # print("followable" + followable_dic[user])
            if author in (followable_dic[user]): # 글 많이 읽은 작가면
                base = base * 2
                # print("followable author " + str(base))
        
        if base > 1.8:
            # print("--- id: " + arc_id + " : " + str(base))
            arc_dic[arc_id] = base
    
    arc_dic = sorted(arc_dic, key = lambda k : arc_dic[k], reverse = True)
    # print(str(arc_dic[:100]))
    
    arc_list = list(arc_dic).copy()
    # print(str(len(arc_list)))
    if len(arc_list) >= 100:
        result_dic[user] = arc_list[:100]
        # print('all list fill')
    else: 
        remain = 100 - len(arc_list)
        result_dic[user] = arc_list
        # print('remain the list: ' + str(remain))

In [ ]:
remain_dic = {}

step = 0
step_end = len(predict_users.user_id)

for user in predict_users.user_id:
    # print(user)
    arc_dic = {}
    
    step +=1
    drawProgressBar(step / step_end)
    
    # 다 채운 유저면 다음으로 넘어가기
    if len(result_dic[user]) == 100:
        continue
    
    is_in_preferdic = False
    if user in user_prefer_dic:
        is_in_preferdic = True
        
    has_follow = False
    if user in user_follow_dict:
        has_follow = True
    
    has_followable = False
    if user in followable_dic:
        has_followable = True
        
    has_mag_dic = False
    if user in user_mag_dic:
        has_mag_dic = True
        
    has_key_dic = False
    if user in user_keywords_dic:
        has_key_dic = True
    
    # print(len(result_dic[user]))
    remain = 100 - len(result_dic[user])
        
    for key, value in pre_atc_dic.items(): # 2월 이후 글 제외하고 다시 추천
        # print(value)
        if key in result_dic[user]:
            continue
        
         # 유저가 이미 읽은글인지 판단
        if key in user_read_dic[user]:
            continue
        
        author = value['author']
        arc_id = key
        date = value['date_point']
        popular = value['popular']
        
        au_prefer = 0.0
        if is_in_preferdic:
            if author in user_prefer_dic[user]:
                au_prefer = user_prefer_dic[user][author] # 작가 선호도
        
        mag_prefer = 0.0
        if has_mag_dic:
            if arc_id in mags_dic:
                mag_id = mags_dic[arc_id]
                if mag_id in user_mag_dic[user]:
                    mag_prefer = user_mag_dic[user][mag_id] # 매거진 선호도
                    # print('mag_id: ' + str(mag_id) + ": " + str(mag_prefer))

        keyword_score = 0.0

        if has_key_dic and arc_id in key_dic:
            keys = key_dic[arc_id]
            # print('key_list: ' + str(keys))
            
            for w in keys:
                if w in user_keywords_dic[user]:
                    keyword_score += user_keywords_dic[user][w]
            
        if popular > 1.0:
            popular = 1.0

        # base = date + au_prefer + popular + mag_prefer + keyword_score
        base = (au_prefer + mag_prefer + keyword_score) * 2 + popular # 개인화 가중치

        # 팔로우 작가가 있는 유저
        if has_follow:
            if author in (user_follow_dict[user]): # 팔로우 작가면
                base = base * 3
                # print("follow author " + str(base))
                
        if has_followable:
            # print("followable" + followable_dic[user])
            if author in (followable_dic[user]): # 글 많이 읽은 작가면
                base = base * 2
                # print("followable author " + str(base))
        
        
        
        if base > 0.3:
            # print("--- id: " + arc_id + " : " + str(base) + ' : ' + str(keyword_score))
            arc_dic[arc_id] = base
    
    arc_dic = sorted(arc_dic, key = lambda k : arc_dic[k], reverse = True)
    # print(arc_dic[:remain])
    
    arc_list = list(arc_dic[:remain]).copy()
    remain_dic[user] = arc_list

In [ ]:
merge_dic = {}

for user in predict_users.user_id:
    feb_recomm = copy.deepcopy(result_dic[user])
    after_recomm = []
    
    if len(feb_recomm) < 100:
        after_recomm = copy.deepcopy(remain_dic[user])
    
    # print(str(len(feb_recomm)) + ':' + str(feb_recomm))
    # print(str(len(after_recomm)) + ':' + str(after_recomm))
    
    merge_dic[user] = feb_recomm
    feb_cnt = 100 - len(feb_recomm)
    
    if not feb_cnt == 0:
        merge_dic[user].extend(after_recomm[:feb_cnt])
    
    # print(str(merge_dic[user]))
    
    if len(merge_dic[user]) != 100: print('ERROR!!!' + user + str(len(merge_dic[user])))
        
print("[Train-1] End")

In [286]:
# 2차 학습
print("[Train-2] Start")
train2_data = atc_read_cnt_nn.sort_values(["read_cnt"], ascending=[False])[(atc_read_cnt_nn.reg_datetime < "2019-02-01")].article_id

c:\users\cmtyx\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [287]:
# append
train2_c = list(train2_data)
train2_p = [0] * len(train2_data)

In [288]:
# 2차 가중치 설정
step = 0
step_end = len(predict_users.user_id)

train2_preds = {}
columns = train2_c.copy()
for id in predict_users.user_id:
    p = train2_p.copy()

    for idx in range(0, len(columns)):
        follow = columns[idx].split('_')[0]
        article_rank = getFollowingRanking(id)
        if follow in user_follow_dict[id]:
            if article_rank.get(follow, "empty") != "empty":
                p[idx] += (article_rank[follow]) # 팔로우 가중치
        p[idx] += getArticleScore(columns[idx]) + getKeywordAvgScore(user_keyword_ranking_dic[id], columns[idx]) 
    
    step += 1
    drawProgressBar(step / step_end)
    
    preds = [i[0] for i in sorted([i for i in zip(columns, p) if i[0] not in user_read_dic[id]], key=lambda i:i [1], reverse=True)[:200]]
    train2_preds[id] = preds
    
print("[Train-2] End")

[====================] 100%

In [51]:
# 중복없이 100개 추출하기
temp_result = {}
result = {}
for id in predict_users.user_id:
    result[id] = merge_dic[id][:30]
    temp_result[id] = [article for article in train2_preds[id] if article not in result[id]]
    result[id].extend(temp_result[id][:70])

NameError: name 'merge_dic' is not defined

In [290]:
# 저장하기
save_data = []
for idx in range(0, len(predict_users.user_id)):
    user_id = predict_users.user_id[idx]
    temp = [user_id]
    temp.extend(result[user_id])
    temp = [' '.join(temp)]  
    save_data.append(temp);

In [291]:
save = pd.DataFrame(save_data)
save.to_csv("recommend.txt", header=False, index=False)
print("[Success!] recommend.txt")